In [ ]:
#Import der benötigten Libraries
import numpy as np
import matplotlib.pyplot as plt
import os
import librosa
import librosa.display
from IPython.display import Audio
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow import keras as k

In [ ]:
os_dir = os.getcwd()
path_to_soundfilefolder = f'{os_dir}\\Datensatz\\processed_audio'
print(path_to_soundfilefolder)

In [ ]:
#Zeug von Librosa
#y_harmonic, y_percussive = librosa.effects.hpss(y)  -------> noch beschreiben, was die einzelnen Attribute aussagen
#tempo, beat_frames = librosa.beat.beat_track(y=y_percussive, sr=sr)
#mfcc = librosa.feature.mfcc(y=y, sr=sr, hop_length=hop_length, n_mfcc=13)
#mfcc_delta = librosa.feature.delta(mfcc)
#beat_mfcc_delta = librosa.util.sync(np.vstack([mfcc, mfcc_delta]), beat_frames)
#chromagram = librosa.feature.chroma_cqt(y=y_harmonic, sr=sr)
#beat_chroma = librosa.util.sync(chromagram, beat_frames, aggregate=np.median)
#beat_features = np.vstack([beat_chroma, beat_mfcc_delta])

In [ ]:
#unsere Attribute
attributes = []

y, sr = librosa.load("C:\\Users\\Luis\\Music_Genre_Git\\Datensatz\\processed_audio\\blues_1_0.wav")
stft_array = librosa.feature.chroma_stft(y=y, sr=sr)
stft_mean = np.mean(stft_array)
stft_var = np.var(stft_array)
cq_array = librosa.feature.chroma_cqt(y=y, sr=sr)
cq_mean = np.mean(cq_array)
cq_var = np.var(cq_array)
rms_array = librosa.feature.rms(y=y)
rms_mean = np.mean(rms_array)
rms_var = np.var(rms_array)
cent_array = librosa.feature.spectral_centroid(y=y, sr=sr)
cent_mean = np.mean(cent_array)
cent_var = np.var(cent_array)
spec_array = librosa.feature.spectral_bandwidth(y=y, sr=sr)
spec_mean = np.mean(spec_array)
spec_var = np.var(spec_array)

attributes.extend([stft_mean, stft_var, rms_mean, rms_var, cent_mean, cent_var, spec_mean, spec_var]) 
print(attributes)

In [ ]:
#Import der Sound-Dateien\n
import os
import numpy as np

#Zuordnung der Labels zu Zahlen
genre_names = []

#enthält sämtliche Einträge
features = []
#enthält die Ergebnisse sämtlicher Einträge
genre = []

print(path_to_soundfilefolder)

for filename in os.listdir(path_to_soundfilefolder):
    #hier kommen alle Attribute pro Soundfile rein
    attributes = []
    
    #Umwandlung der Soundfiles in repräsentative Zahlenwerte
    print(f"Converting: {filename}")
    y, sr = librosa.load(f'{os_dir}\\Datensatz\\processed_audio\\' + filename)
    
    #extrahieren der aussagekräftigen Eigenschaften
    stft_array = librosa.feature.chroma_stft(y=y, sr=sr)
    stft_mean = np.mean(stft_array)
    stft_var = np.var(stft_array)
    cq_array = librosa.feature.chroma_cqt(y=y, sr=sr)
    cq_mean = np.mean(cq_array)
    cq_var = np.var(cq_array)
    rms_array = librosa.feature.rms(y=y)
    rms_mean = np.mean(rms_array)
    rms_var = np.var(rms_array)
    cent_array = librosa.feature.spectral_centroid(y=y, sr=sr)
    cent_mean = np.mean(cent_array)
    cent_var = np.var(cent_array)
    spec_array = librosa.feature.spectral_bandwidth(y=y, sr=sr)
    spec_mean = np.mean(spec_array)
    spec_var = np.var(spec_array)
    
    #anhängen
    attributes.extend([stft_mean, stft_var, rms_mean, rms_var, cent_mean, cent_var, spec_mean, spec_var])    
    #Anhängen eines Eintrages
    features.append(attributes)
    
    #Erzeugen der Outputs
    genre_name = filename.split("_")[0]
    if genre_name not in genre_names:
        genre_names.append(genre_name)
    for i in range(len(genre_names)):
        if genre_name == genre_names[i]:
            genre.append(i)
            print(genre_name,i)
            break

In [ ]:
genre_list = np.array(genre)
feature_list = np.array(features)

print(len(genre_list))
print(len(feature_list))

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(feature_list,genre_list,test_size = 0.2,random_state = 42)

print(len(x_train))
print(y_train)


In [ ]:
print(x_train[0])
print(y_train[0])

In [ ]:
#Programmierung des Modells
import tensorflow as tf
from tensorflow import keras

model = keras.Sequential([
    keras.layers.Dense(64, activation=tf.nn.relu),
    keras.layers.Dense(32, activation=tf.nn.relu),
    keras.layers.Dense(16, activation=tf.nn.relu),
    keras.layers.Dense(10, activation=tf.nn.softmax)
])

model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

model.fit(x_train, y_train, epochs=300)
model.summary()

In [ ]:
#bestes Modell aus dem Internet
model1 = k.models.Sequential([
    k.layers.Dense(1024, activation='relu', input_shape=(x_train.shape[1],)),
    k.layers.Dropout(0.3),
    
    k.layers.Dense(512, activation='relu'),
    k.layers.Dropout(0.3),

    k.layers.Dense(256, activation='relu'),
    k.layers.Dropout(0.3),

    k.layers.Dense(128, activation='relu'),
    k.layers.Dropout(0.3),

    k.layers.Dense(64, activation='relu'),
    k.layers.Dropout(0.3),

    k.layers.Dense(10, activation='softmax'),
])
print(model1.summary())

model1.compile(optimizer='rmsprop',
                  loss='sparse_categorical_crossentropy',
                  metrics='accuracy')

model1.fit(x_train, y_train, epochs=300)

In [ ]:
import numpy as np
import sklearn.metrics

predictions_confidences = model.predict(x_test)

predictions = []
for entry in predictions_confidences:
        predictions.append(np.argmax(entry))
print(predictions)

In [ ]:
print(f"{sklearn.metrics.classification_report(y_test, predictions)}\n")

In [ ]:
#test librosa
y, sr = librosa.load(f'{os_dir}\\Datensatz\\processed_audio\\blues_1_0.wav')
print(y, sr)

In [ ]:
tempo, beat_frames = librosa.beat.beat_track(y=y, sr=sr)
print(tempo, beat_frames)